In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [96]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd


def get_jobs(num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=software+engineer&sc.keyword=software+engineer&locT=&locId=&jobType="
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Going through each job in this page 
        job_buttons = driver.find_elements_by_class_name("jobContainer")  #jl for Job Listing. These are the buttons we're going to click.
        
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(5)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [97]:
df = get_jobs(200, False, path = '/Users/damanheer/Downloads/chromedriver',slp_time = 15)

 x out worked
Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000
Progress: 4/1000
Progress: 5/1000
Progress: 6/1000
Progress: 7/1000
Progress: 8/1000
Progress: 9/1000
Progress: 10/1000
Progress: 11/1000
Progress: 12/1000
Progress: 13/1000
Progress: 14/1000
Progress: 15/1000
Progress: 16/1000
Progress: 17/1000
Progress: 18/1000
Progress: 19/1000
Progress: 20/1000
Progress: 21/1000
Progress: 22/1000
Progress: 23/1000
Progress: 24/1000
Progress: 25/1000
Progress: 26/1000
Progress: 27/1000
Progress: 28/1000
Progress: 29/1000
 x out failed
Progress: 30/1000
Progress: 31/1000
Progress: 32/1000
Progress: 33/1000
Progress: 34/1000
Progress: 35/1000
Progress: 36/1000
Progress: 37/1000
Progress: 38/1000
Progress: 39/1000
Progress: 40/1000
Progress: 41/1000
Progress: 42/1000
Progress: 43/1000
Progress: 44/1000
Progress: 45/1000
Progress: 46/1000
Progress: 47/1000
Progress: 48/1000
Progress: 49/1000
Progress: 50/1000
Progress: 51/1000
Progress: 52/1000
Progress: 53/1000
Progress: 

ElementClickInterceptedException: Message: element click intercepted: Element <div class="jobContainer">...</div> is not clickable at point (240, 699). Other element would receive the click: <div class="gdGrid noPad">...</div>
  (Session info: chrome=81.0.4044.138)


In [84]:
df

,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership
0,OneSpan\n3.8,"Gemalto, RSA Security, Entrust Datacard",1991,"Chicago, IL",Computer Hardware & Software,"About OneSpan\n\nAt OneSpan, you will be part ...",Software Integration Engineer,"Dallas, TX",3.8,$100 to $500 million (USD),$88K-$106K (Glassdoor est.),Information Technology,501 to 1000 employees,Company - Public
1,Computer Services\n4.0,-1,1965,"Paducah, KY",Financial Transaction Processing,Job Description\n\nCSI’s Payments Software Eng...,Software Engineer,"Valparaiso, IN",4.0,$100 to $500 million (USD),$88K-$106K (Glassdoor est.),Finance,1001 to 5000 employees,Company - Public
2,Kryterion Inc\n3.0,"Pearson VUE, Prometric, PSI Services",2001,"Phoenix, AZ",IT Services,Software Engineer II\n\nThis Software Engineer...,Software Engineer II,"Phoenix, AZ",3.0,$10 to $25 million (USD),$88K-$106K (Glassdoor est.),Information Technology,51 to 200 employees,Company - Private
3,American Axle & Manufacturing\n3.3,-1,1994,"Detroit, MI",Transportation Equipment Manufacturing,We are AAM. We have the POWER to move the worl...,Software Engineer,"Detroit, MI",3.3,$5 to $10 billion (USD),$88K-$106K (Glassdoor est.),Manufacturing,10000+ employees,Company - Public
4,Profi Vision,-1,-1,"Allentown, PA",-1,Profi-Vision is a system integration company w...,Controls and Software Engineer,"Allentown, PA",-1,$1 to $5 million (USD),$88K-$106K (Glassdoor est.),-1,1 to 50 employees,Company - Private
5,Buildium\n4.8,-1,2004,"Boston, MA",Computer Hardware & Software,"Buildium is looking for smart, driven, enthusi...",Senior Software Engineer,"Boston, MA",4.8,Unknown / Non-Applicable,$88K-$106K (Glassdoor est.),Information Technology,51 to 200 employees,Company - Private
6,Bectran\n2.7,-1,2010,"Schaumburg, IL",Computer Hardware & Software,"Who We Are:\nBectran, Inc. was founded in 2010...",Software Engineer,"Schaumburg, IL",2.7,$10 to $25 million (USD),$88K-$106K (Glassdoor est.),Information Technology,51 to 200 employees,Company - Private
7,SecureTrust,-1,2019,"Chicago, IL",Security Services,"SecureTrust, a Trustwave division, leads the i...",Software Engineer,"Chicago, IL",-1,$100 to $500 million (USD),$88K-$106K (Glassdoor est.),Business Services,201 to 500 employees,Company - Private
8,ReCharge Payments\n4.4,-1,2015,"Santa Monica, CA",Enterprise Software & Network Solutions,"Overview\n\n\nWith over 10,000 online merchant...",Software Engineer,"Phoenix, AZ",4.4,Unknown / Non-Applicable,$88K-$106K (Glassdoor est.),Information Technology,51 to 200 employees,Company - Private
9,Remedi SeniorCare\n3.4,-1,2002,"Baltimore, MD",Health Care Services & Hospitals,Remedi SeniorCare is seeking a highly qualifie...,Software Development Engineer,"Troy, OH",3.4,$100 to $500 million (USD),$88K-$106K (Glassdoor est.),Health Care,1001 to 5000 employees,Company - Private


In [83]:
df.to_csv('/Users/damanheer/Downloads/data.csv')